In [42]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pathlib
import random
import pandas as pd

from credentials import CONNECTION_INFO
from constants import *

import encoders
import db_connect
import helpers

## Config
This notebook has a lot of options to adjust, most of which are controlled here.

In [102]:
ENCODER = encoders.ENCODER_CAESAR
CHUNK_SIZE = 512
PROCESSING_UNITS = CHUNK_SIZE // 4
LAYER_UNITS = max(1, CHUNK_SIZE // 10)

EXTRA_CHECKS = True # Whether to run some (potentially slow) debug checks

INFER_TEXT = False
INFER_KEY = not INFER_TEXT

USE_CUSTOM_LOSS = True
USE_CUSTOM_OUTPUT_ACTIVATION = True

OUTPUT_MIN = 0
OUTPUT_MAX = len(encoders.CHARSET)-1
CUSTOM_LOSS_MODULO = OUTPUT_MAX+1

if INFER_TEXT:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mean_squared_error"
    OUTPUT_SIZE = CHUNK_SIZE
    OPTIMIZER = "sgd"
else:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mae"
    OPTIMIZER = "adamax"    

    if ENCODER == encoders.ENCODER_CAESAR:
        OUTPUT_SIZE = 1
    elif ENCODER == encoders.ENCODER_SUBST:
        OUTPUT_SIZE = len(encoders.CHARSET)
    else:
        raise Exception(f"Unsupported encoder {ENCODER}")

ENCRYPTED_FILE_LIMIT = -1 # -1 to disable limit

BASE_TRAIN_PCT = 0.75   # Start here. If train or test count would exceed the max, reduce it. Note 0.75 is the default.
MAX_TRAIN_COUNT = 100 # -1 to disable
MAX_TEST_COUNT =  100 # -1 to disable
SPLIT_SEED = 42

LOAD_BEST_MODEL = False # If False, a new model will be created from scratch
SAVE_BEST_MODEL = True
BEST_PATH = './saved_models/best.keras'

# Whether to run the tuner or the hard-coded network build code
TUNE_NETWORK = False
BUILD_NETWORK = not TUNE_NETWORK
TRAIN_MODEL = BUILD_NETWORK

TUNER_DIRECTORY = "tuner_projects"
TUNER_PROJECT_NAME = "KT"

EPOCHS = 5
BATCH_SIZE = 256 # Default is 32 -- going higher speeds things up a LOT, but may cause memory problems

# Data Retrieval and Structuring

In [44]:
# Get database IDs for encoders and key types

encoder_ids= {}
key_type_ids = {}

db = db_connect.DB(CONNECTION_INFO)
with db.get_session() as session:
    for encoder in encoders.ALL_ENCODER_NAMES:
        id = db.get_encoder_id(session, encoder)
        encoder_ids[encoder] = id

    print(f"Encoder IDs: {encoder_ids}")

    for key_type in encoders.KEY_NAMES:
        id = db.get_key_type_id(session, key_type)
        key_type_ids[key_type] = id

    print(f"Key Type IDs: {key_type_ids}")

Encoder IDs: {'None': 1, 'Simplifier': 2, 'Caesar Cipher': 3, 'Substitution Cipher': 4}
Key Type IDs: {'Character Offset': 1, 'Character Map': 2}


In [45]:
# Map source ID to plaintext file (1) details, and source ID to corresponding ciphertext files (1+) details
sid_to_p = {}
sid_to_c = {}

cipher_id = encoder_ids[ENCODER]
with db.get_session() as session:
    # Get all files encrypted with the cipher we care about
    encrypted_files = db.get_files_by_source_and_encoder(session, -1, cipher_id)

    if len(encrypted_files) > ENCRYPTED_FILE_LIMIT and ENCRYPTED_FILE_LIMIT > 0:
        print(f"Found {len(encrypted_files)} encrypted files")
        encrypted_files = random.sample(encrypted_files, ENCRYPTED_FILE_LIMIT)
    print(f"Using {len(encrypted_files)} encrypted files")

    for c in encrypted_files:
        sid = c.source_id
    
        if sid not in sid_to_p:
            plaintext_ids = db.get_files_by_source_and_encoder(session, sid, encoder_ids[encoders.ENCODER_SIMPLIFIER])
            if len(plaintext_ids) != 1:
                raise Exception(f"Found {len(plaintext_ids)} plaintexts for source ID {sid}; should be exactly 1")
            sid_to_p[sid] = plaintext_ids[0]

        if sid not in sid_to_c:
            sid_to_c[sid] = []
        sid_to_c[sid].append(c)

len(sid_to_p), len(sid_to_c)

Using 114 encrypted files


(19, 19)

In [46]:
# Build up the features (X, the cipher texts as offsets) and targets (y, either the plain texts as offsets OR the key).
# Note targets are not necessarily unique.
X = []
y = []

with db.get_session() as session:
    for sid in sid_to_p:
        if INFER_TEXT:
            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
            target_chunks = helpers.chunkify(plaintext, CHUNK_SIZE)    
    
        for c in sid_to_c[sid]:
            ciphertext = encoders.string_to_offsets(helpers.read_text_file(c.path))
            feature_chunks = helpers.chunkify(ciphertext, CHUNK_SIZE)

            if INFER_KEY:                
                if ENCODER == encoders.ENCODER_CAESAR:
                    key_value = float(db.get_key_by_id(session, c.key_id).value)

                    if EXTRA_CHECKS:
                        # Decode with the key we got from the DB, make sure it actually works
                        CHECK_CHANCE = 0.1
                        if random.random() < CHECK_CHANCE:
                            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
                            plainttext_str = encoders.offsets_to_string(plaintext)
                            ciphertext_str = encoders.offsets_to_string(ciphertext)
                            decoded_str = encoders.decode_caesar(ciphertext_str, int(key_value))
                            if decoded_str != plainttext_str:                                                    
                                print(decoded_str == plainttext_str)
                                print(decoded_str[0:128], plainttext_str[0:128])
                                raise Exception("Decode error")
                    
                elif ENCODER == encoders.ENCODER_SUBST:
                    key_str = db.get_key_by_id(session, c.key_id).value
                    key_value_ints = encoders.string_to_offsets(key_str)
                    key_value = np.array(key_value_ints).astype(float)

                else:
                    raise Exception(f"Unsupported encoder {ENCODER}")
        
            for i in range (len(feature_chunks)):
                X.append(np.array(feature_chunks[i]).astype(float))

                if INFER_TEXT:
                    y.append(np.array(target_chunks[i]).astype(float))

                if INFER_KEY:
                    y.append(key_value)

X = np.array(X)
y = np.array(y)

X.shape, y.shape

((69396, 512), (69396,))

In [47]:
# Split the preprocessed data into a training and testing dataset
train_count = int(round(len(y) * BASE_TRAIN_PCT))
if train_count > MAX_TRAIN_COUNT and MAX_TRAIN_COUNT > -1:
    print(f"Train count would be {train_count}")
    train_count = int(MAX_TRAIN_COUNT)
print(f"Train count is {train_count}")

test_count = len(y) - train_count
if test_count > MAX_TEST_COUNT and MAX_TEST_COUNT > -1:
    print(f"Test count would be {test_count}")
    test_count = int(MAX_TEST_COUNT)
print(f"Test count is {test_count}")

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_count, test_size=test_count, random_state=SPLIT_SEED)
print( "Initial counts: ", len(X), len(y), len(X_train), len(X_test), len(y_train), len(y_test) )

Train count would be 52047
Train count is 100
Test count would be 69296
Test count is 100
Initial counts:  69396 69396 100 100 100 100


In [48]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((100, 512), (100, 512))

# Tensorflow Callbacks
Custom layer activation function, loss and accuracy functions, and model save checkpoint.

In [109]:
tf.keras.utils.get_custom_objects().clear()

# Custom output activation, forcing output to be within range -- but not rounding it off
# Possibly not needed, sigmoid + rescaler might work just as well
@tf.keras.utils.register_keras_serializable(package="RPM_breakerbot", name="modulo_output")
def modulo_output(x):
    return tf.math.mod(x, OUTPUT_MAX)

# Custom loss function, adapted from code generated by Copilot
@tf.keras.utils.register_keras_serializable(package="RPM_breakerbot", name="modulo_distance_loss")
def modulo_distance_loss(y_true, y_pred):
    """ Custom loss function to compute the modulo distance. 
    Args: 
        y_true: True values (ground truth). 
        y_pred: Predicted values. 
        modulo: The modulo value to apply -- hard coded.
    Returns: 
        The computed loss. 
    """ 
    # Compute the raw difference
    diff = tf.abs(y_true - y_pred)
    # Apply modulo operation to handle wrap-around cases
    mod_diff = tf.math.mod(diff, CUSTOM_LOSS_MODULO)
    # Ensure the distance is within the range [0, CUSTOM_LOSS_MODULO/2]
    loss = tf.minimum(mod_diff, CUSTOM_LOSS_MODULO - mod_diff) 
    return tf.reduce_mean(loss)

# Custom accuracy function, counterpart to the loss function above.
# Returns accuracy as 1 - (average percent distance from correct value)
@tf.keras.utils.register_keras_serializable(package="RPM_breakerbot", name="modulo_distance_accuracy")
def modulo_distance_accuracy(y_true, y_pred):
    diff = tf.abs(y_true - y_pred)
    mod_diff = tf.math.mod(diff, CUSTOM_LOSS_MODULO)
    loss = tf.minimum(mod_diff, CUSTOM_LOSS_MODULO - mod_diff)

    good_part = tf.math.subtract(CUSTOM_LOSS_MODULO / 2, loss)
    accuracy = tf.math.divide(good_part, CUSTOM_LOSS_MODULO / 2)

    return tf.reduce_mean(accuracy)

# Custom accuracy, percent of correct values after rounding and doing modulo division
@tf.keras.utils.register_keras_serializable(package="RPM_breakerbot", name="modulo_rounded_accuracy")
def modulo_rounded_accuracy(y_true, y_pred):
    # y_true SHOULD all be round, in-bounds numbers but just in case...
    true_rounded = tf.math.round(y_true)
    true_mod = tf.math.mod(true_rounded, CUSTOM_LOSS_MODULO)

    # y_pred came straight from the model, so it needs to be rounded and mod'ed
    pred_rounded = tf.math.round(y_pred)
    pred_mod = tf.math.mod(pred_rounded, CUSTOM_LOSS_MODULO)

    # Count matches, as a percentage by averaging all the 0's and 1's
    matches_bool = tf.math.equal(true_mod, pred_mod)
    matches_float = tf.cast(matches_bool, tf.float64)
    return tf.reduce_mean(matches_float)

In [50]:
if EXTRA_CHECKS:
    # Testing my loss and accuracy functions
    t_true = [[1.0, 2.0, 3.0, CUSTOM_LOSS_MODULO*5]]*2
    t_pred = [[0.4, 1.5, 3.5, CUSTOM_LOSS_MODULO + 0.4]]*2
    
    t_true_ts = tf.constant(np.array(t_true).astype(float))
    t_pred_ts = tf.constant(np.array(t_pred).astype(float))
    loss = modulo_distance_loss(t_true_ts, t_pred_ts)
    accD = modulo_distance_accuracy(t_true_ts, t_pred_ts)
    accR = modulo_rounded_accuracy(t_true_ts, t_pred_ts)
    print("true:", t_true_ts)
    print("pred:", t_pred_ts)
    print("rond:", tf.math.round(t_pred_ts))
    print("diff", abs(t_true_ts - t_pred_ts))
    print("loss:", loss)
    print("accD:", accD)
    print("accR:", accR)


true: tf.Tensor(
[[ 1.  2.  3. 50.]
 [ 1.  2.  3. 50.]], shape=(2, 4), dtype=float64)
pred: tf.Tensor(
[[ 0.4  1.5  3.5 10.4]
 [ 0.4  1.5  3.5 10.4]], shape=(2, 4), dtype=float64)
rond: tf.Tensor(
[[ 0.  2.  4. 10.]
 [ 0.  2.  4. 10.]], shape=(2, 4), dtype=float64)
diff tf.Tensor(
[[ 0.6  0.5  0.5 39.6]
 [ 0.6  0.5  0.5 39.6]], shape=(2, 4), dtype=float64)
loss: tf.Tensor(0.49999999999999967, shape=(), dtype=float64)
accD: tf.Tensor(0.9000000000000001, shape=(), dtype=float64)
accR: tf.Tensor(0.5, shape=(), dtype=float64)


# Hyperband Tuning

# Model Reload /Creation

In [110]:
if BUILD_NETWORK:
    print("Building new model")
    nn = tf.keras.models.Sequential()

    # Input layer
    nn.add(tf.keras.layers.Embedding(input_dim=CHUNK_SIZE, output_dim=PROCESSING_UNITS, name="Embedding_Input"))

    # This LSTM layer seems to do most of the real work
    nn.add(tf.keras.layers.LSTM(PROCESSING_UNITS, name="LSTM"))

    # Sigmoid layer produces an output between 0 and 1
    nn.add(tf.keras.layers.Dense(units=PROCESSING_UNITS, activation="sigmoid", name="Sigmoid"))

    # Rescale that 0-1 value from Sigmoid to the correct output range
    nn.add(tf.keras.layers.Rescaling(scale=OUTPUT_MAX, offset=0, name="Rescaler")) # Input is 0-1

    # Do modulo division to enforce output range limit
    # Note this seems like it should be totally redundant. But the Tuner results suggest that including both
    # mechanisms (Sigmoid + Rescaling, and Modulo Division) produces better results. I don't know why.
    nn.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation=modulo_output, name="Output_Limiter"))

    # Check the structure of the model
    nn.summary()

Building new model


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Embedding_Input (Embedding)     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Sigmoid (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Rescaler (Rescaling)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Limiter (Dense)          │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Model Training

In [100]:
%%time

# It can be helpful to load the best, train some more, and try to improve it:
if LOAD_BEST_MODEL:
    if os.path.exists(BEST_PATH):
        print(f"Loading model from {BEST_PATH}")
        nn = tf.keras.models.load_model(BEST_PATH,
            custom_objects={
                'modulo_distance_loss': modulo_distance_loss,
                'modulo_distance_accuracy': modulo_distance_accuracy,
                'modulo_rounded_accuracy': modulo_rounded_accuracy,
                'modulo_output': modulo_output
        })

if TRAIN_MODEL:
    # Decide what metrics to use
    if USE_CUSTOM_LOSS:
        loss = modulo_distance_loss
        metrics = [modulo_distance_accuracy, modulo_rounded_accuracy]
        save_best_monitor = "modulo_distance_accuracy"
        save_best_mode = "max"
    else:
        loss = LOSS_METRIC
        metrics = [MAIN_ACCURACY_METRIC]
        save_best_monitor = "loss"
        save_best_mode = "min"

    # Training checkpoint to save after each epoch, if it is a new best model:    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=BEST_PATH,
        monitor=save_best_monitor,
        mode=save_best_mode,
        save_best_only=True,
        save_weights_only=False,
        verbose=2)

    print(f"Training model")
    
    if SAVE_BEST_MODEL:
        callbacks = [model_checkpoint_callback]
    else:
        callbacks = None
    
    # Compile the Sequential model together and customize metrics
    nn.compile(loss=loss, optimizer=OPTIMIZER, metrics=metrics)
    
    # Fit the model to the training data
    fit_model = nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=BATCH_SIZE)

nn.summary()

ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=./saved_models/best.hd5

In [59]:
# Evaluate the model using the test data

if USE_CUSTOM_LOSS and INFER_KEY:    
    print("Evaluating with model.predict() ...")
    raw_pred = nn.predict(X_test_scaled, batch_size=BATCH_SIZE)

    # My custom loss and accuracy functions are running out of memory for some reason...
    if False:
        y_pred = tf.constant(np.array(raw_pred).astype(np.float64))
        loss = modulo_distance_loss(y_test, y_pred)
        accuracy_distance = modulo_distance_accuracy(y_test, y_pred)
        accuracy_rounded = modulo_rounded_accuracy(y_test, y_pred)
        print(f"Loss: {loss:0.6}, Accuracy (Distance): {accuracy_distance:0.6}, Accuracy (Rounded): {accuracy_rounded:0.6}")

    if INFER_KEY:
        pred_pd = pd.DataFrame(raw_pred)
        print("Inferred key distribution:\n", pred_pd.describe())

print("Evaluating with model.evaluate() ...")
eval_results = nn.evaluate(X_test_scaled, y_test, verbose=2, batch_size=BATCH_SIZE)
print(f"With X_test_scaled, Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")

if EXTRA_CHECKS:
    # Sometimes the results with totally wacky inputs are so similar to the official "test" data,
    # that it's clearly just plain broken.
    eval_results = nn.evaluate(X_train_scaled, y_train, verbose=2, batch_size=BATCH_SIZE)
    print(f"With X_train_scaled, Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")
    
    eval_results = nn.evaluate(X_test, y_test, verbose=2, batch_size=BATCH_SIZE)
    print(f"With X_test,        Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")
    
    eval_results = nn.evaluate(X_test_scaled, y_train[0:len(X_test_scaled)], verbose=2, batch_size=BATCH_SIZE)
    print(f"With mismatch,        Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")

Evaluating with model.predict() ...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Inferred key distribution:
                 0
count  100.000000
mean     3.939304
std      3.242853
min      0.003549
25%      2.541276
50%      2.992238
75%      8.579894
max      8.998241
Evaluating with model.evaluate() ...
1/1 - 0s - 89ms/step - loss: 1.4826 - modulo_distance_accuracy: 0.5067 - modulo_rounded_accuracy: 0.1292
With X_test_scaled, Loss: 1.482580542564392, Accuracy: [0.5067338943481445, 0.12919999659061432]
1/1 - 0s - 89ms/step - loss: 1.1866 - modulo_distance_accuracy: 0.5290 - modulo_rounded_accuracy: 0.1202
With X_train_scaled, Loss: 1.1865808963775635, Accuracy: [0.5290083289146423, 0.12020000070333481]
1/1 - 0s - 82ms/step - loss: 2.8066 - modulo_distance_accuracy: 0.4411 - modulo_rounded_accuracy: 0.0980
With X_test,        Loss: 2.806638479232788, Accuracy: [0.44108685851097107, 0.09799999743700027]
1/1 - 0s - 75ms/step - loss: 2.5769 - modulo_distance_accuracy: 0.5275 - modulo_rounded_ac

# Model Usefulness Spot-Check

In [65]:
# These only really work for the Caesar Cipher

def decode_chunks_with_model(chunks: list[list], model, scaler, input_already_scaled = True) -> list[list]:
    if input_already_scaled:
        return model.predict(chunks)
    else:
        return model.predict(scaler.transform(chunks))

def decode_text_with_model(ciphertext: str, model, scaler) -> str:
    offset_chunks = helpers.chunkify(encoders.string_to_offsets(ciphertext), CHUNK_SIZE)
    decoded_chunks = decode_chunks_with_model(offset_chunks, model, scaler, input_already_scaled = False)
    rounded = np.rint(decoded_chunks.flatten()).astype(int)
    return encoders.offsets_to_string(rounded)

def infer_key_with_model(ciphertext: str, model, scaler) -> int:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    keys = model.predict(scaler.transform(chunks))
    key = int(round(np.median(keys)))
    return key

if INFER_TEXT:    
    CHUNKS_TO_CHECK = 2
else:
    CHUNKS_TO_CHECK = 20

cipher_file_db = sid_to_c[list(sid_to_c.keys())[0]][0]
ciphertext_path = cipher_file_db.path
ciphertext = helpers.read_text_file(ciphertext_path)
ciphertext = ciphertext[0:CHUNK_SIZE * CHUNKS_TO_CHECK]

if INFER_TEXT:    
    print("Decoded   : ", decode_text_with_model(ciphertext, nn, X_scaler))
if INFER_KEY:
    with db.get_session() as session:
        correct_key = db.get_key_by_id(session, cipher_file_db.key_id).value
    print("Correct Key: ", correct_key)
    
    inferred_key = infer_key_with_model(ciphertext, nn, X_scaler)
    print("Inferred Key: ", inferred_key)
    
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    print(nn.predict(scaler.transform(chunks)))
    print(pd.DataFrame(nn.predict(scaler.transform(chunks))).describe())

Correct Key:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Inferred Key:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[[0.72789764]
 [0.78387547]
 [0.7575598 ]
 [0.7087898 ]
 [0.9139681 ]
 [0.7973223 ]
 [0.7717514 ]
 [0.8614311 ]
 [0.8486271 ]
 [0.9378185 ]
 [1.0019217 ]
 [0.9320793 ]
 [0.9229889 ]
 [0.7152004 ]
 [0.7961168 ]
 [0.7729435 ]
 [0.76924133]
 [0.86238384]
 [0.8199539 ]
 [0.84451294]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
               0
count  20.000000
mean    0.827319
std     0.082103
min     0.708790
25%     0.771124
50%     0.808638
75%     0.875280
max     1.001922
